In [34]:
import rqdatac as rqd
import rqoptimizer as rqo
import pandas as pd
import numpy as np
rqd.init()

F:\anaconda\envs\ricequant\lib\site-packages\rqdatac\client.py:190: UserWarning: rqdatac is already inited. Settings will be changed.
  warnings.warn("rqdatac is already inited. Settings will be changed.", stacklevel=0)


In [28]:
def get_target_stock(date,benchmark):
    """选取当期中证500融券股票"""
    # 融券股票列表
    margin_stocks = rqd.get_margin_stocks(date=date,exchange=None,margin_type='stock')
     # 中证500成分股
    component = rqd.index_components(benchmark, date)
    # 取交集
    target_stock = list(set(component).intersection(margin_stocks))
    return target_stock


def get_target_portfolio(target_stock,date,benchmark):
    """按照产品设置, 计算目标组合"""
    # 定义目标函数和约束条件
    objective = rqo.MinTrackingError()
    
    # 行业中性
    industry_constraints = rqo.WildcardIndustryConstraint(
                                exclude=None,
                                lower_limit=-0.1,
                                upper_limit=0.1,
                                relative=True,
                                hard=True,
                                classification=rqo.IndustryClassification.ZX)
    # 风格约束
    style_constraints = rqo.StyleConstraint(
                                ['size','beta','non_linear_size'], 
                                lower_limit=-0.1, 
                                upper_limit=0.1, 
                                relative=True, 
                                hard=True)
    
    constraints = [industry_constraints, style_constraints]
    # 个股权重
    bnds = {}
    index_weights = rqd.index_weights(benchmark,date)
    for code in target_stock:
        stock_weight = index_weights.loc[code]
        bnds[code] = (max(0,stock_weight-0.05),stock_weight+0.05)
    
    target_portfolio = rqo.portfolio_optimize(target_stock,date, 
                                       benchmark=benchmark, 
                                       objective=objective,
                                       cons=constraints,
                                       bnds=bnds,
                                       cov_model=rqo.const.CovModel.FACTOR_MODEL_MONTHLY) 
    # 由于精度问题, 优化结果可能包含非常接近0的股票权重, 在此直接舍去
    return target_portfolio.loc[lambda x:x!=0]

In [29]:
date = '20200203'
benchmark = '000905.XSHG'

In [30]:
target_stock = get_target_stock(date,benchmark)
target_portfolio = get_target_portfolio(target_stock,date,benchmark)

WARN: aa_init returned NULL, no acceleration applied.


In [31]:
target_portfolio

000006.XSHE    0.001102
000008.XSHE    0.001605
000009.XSHE    0.003867
000012.XSHE    0.001523
000021.XSHE    0.002574
                 ...   
603882.XSHG    0.002342
603883.XSHG    0.001950
603885.XSHG    0.001829
603888.XSHG    0.001589
603939.XSHG    0.003015
Length: 463, dtype: float64

In [35]:
def get_tracking_error(weight,date,benchmark):
    """
    weight: 现在持仓权重series
    """
    date_previous = rqd.get_previous_trading_date(date)
    benchmark_weight = rqd.index_weights(benchmark,date_previous)
    w = (weight-benchmark_weight).fillna(0)
    # 因子暴露
    X = rqd.get_factor_exposure(w.index, date_previous, date_previous).droplevel(0)
    # 因子协方差矩阵
    C = rqd.get_factor_covariance(date_previous, horizon= 'daily')
    return np.sqrt(w.T.dot(X).dot(C).dot(X.T).dot(w))

In [37]:
get_tracking_error(target_portfolio,date,benchmark)

0.00847222262815597

In [38]:
target_portfolio_0 = pd.Series(1/len(target_portfolio),index = target_portfolio.index)

In [40]:
get_tracking_error(target_portfolio_0,date,benchmark)

0.026845424650056394

In [17]:
date_previous = rqd.get_previous_trading_date(date)
date_previous

datetime.date(2020, 1, 23)

In [20]:
benchmark_weight = rqd.index_weights(benchmark,date_previous)
benchmark_weight

order_book_id
000006.XSHE    0.00109
000008.XSHE    0.00152
000009.XSHE    0.00381
000012.XSHE    0.00147
000021.XSHE    0.00250
                ...   
603712.XSHG    0.00129
002957.XSHE    0.00028
601615.XSHG    0.00079
601975.XSHG    0.00123
601598.XSHG    0.00148
Length: 500, dtype: float64

In [23]:
w = (target_portfolio-benchmark_weight).fillna(0)
w

000006.XSHE    0.000127
000008.XSHE    0.000132
000009.XSHE    0.000083
000012.XSHE   -0.000004
000021.XSHE    0.000005
                 ...   
603883.XSHG   -0.000007
603885.XSHG    0.000060
603888.XSHG    0.000221
603939.XSHG    0.000061
603983.XSHG    0.000000
Length: 500, dtype: float64

In [28]:
x = rqd.get_factor_exposure(w.index, date_previous, date_previous, factors = None,industry_mapping = True )

In [35]:
x = x.droplevel(0)

In [36]:
x

,book_to_price,comovement,liquidity,growth,non_linear_size,beta,leverage,residual_volatility,momentum,earnings_yield,...,建筑装饰,电气设备,国防军工,计算机,传媒,通信,银行,非银金融,汽车,机械设备
order_book_id,,,,,,,,,,,,,,,,,,,,,
000006.XSHE,0.475793,1,-0.295875,0.307792,0.755315,0.116549,1.089844,-1.241484,-0.799015,1.001859,...,0,0,0,0,0,0,0,0,0,0
000008.XSHE,0.060945,1,0.111157,0.006687,0.734864,-0.058254,-0.894119,-0.733240,-1.152876,-0.030005,...,0,0,0,0,0,0,0,0,0,1
000009.XSHE,-0.201101,1,1.732982,0.120170,0.334504,0.631078,1.329042,1.211328,0.454467,0.606678,...,0,0,0,0,0,0,0,0,0,0
000012.XSHE,0.672089,1,0.042244,0.159624,0.746070,0.841221,-0.101125,-0.982249,-0.094201,-0.065994,...,0,0,0,0,0,0,0,0,0,0
000021.XSHE,-0.857051,1,1.890076,0.591904,0.061032,1.609757,-0.259200,0.955690,1.419319,-0.276818,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603883.XSHG,-1.104670,1,-0.343791,0.025049,0.102401,-0.848131,0.199681,0.490998,-0.430741,-0.166845,...,0,0,0,0,0,0,0,0,0,0
603885.XSHG,-0.487789,1,-0.638340,0.197422,-0.110368,0.481707,1.131194,-0.388642,-0.062186,0.033240,...,0,0,0,0,0,0,0,0,0,0
603888.XSHG,-1.066582,1,1.398533,-0.487004,0.467213,2.620383,-0.531783,1.631897,-0.018520,-0.336713,...,0,0,0,0,1,0,0,0,0,0


In [30]:
c = rqd.get_factor_covariance(date_previous, horizon= 'daily')

In [31]:
c

,book_to_price,纺织服装,汽车,非银金融,residual_volatility,家用电器,轻工制造,休闲服务,growth,化工,...,beta,leverage,通信,电子,钢铁,商业贸易,采掘,公用事业,综合,建筑材料
book_to_price,0.000837,-0.000251,-0.000178,0.000316,7.980568e-05,0.000161,-1.261082e-04,-0.000235,-0.000098,-0.000042,...,-0.000003,0.000010,-0.000280,-0.001589,0.000665,-0.000081,0.000514,-0.000088,0.000226,0.000511
纺织服装,-0.000251,0.013454,0.001300,-0.003913,-4.747836e-04,0.002214,3.606098e-03,0.005025,0.000071,0.001223,...,-0.000750,0.000127,-0.001102,-0.002292,-0.000522,0.002515,-0.001129,0.000440,0.001219,0.003054
汽车,-0.000178,0.001300,0.015782,-0.005089,1.895463e-04,0.001696,1.539618e-03,0.002811,0.000018,0.001445,...,-0.000027,0.000051,-0.000496,0.002086,-0.000977,0.000917,0.000762,0.000492,0.000311,-0.000343
非银金融,0.000316,-0.003913,-0.005089,0.018730,-3.903368e-04,-0.002784,-3.242662e-03,-0.005134,-0.000010,-0.002833,...,0.000748,0.000106,-0.001972,-0.005193,-0.000649,-0.002074,-0.001922,-0.000973,-0.000534,-0.002105
residual_volatility,0.000080,-0.000475,0.000190,-0.000390,1.523343e-03,-0.000262,6.591435e-07,-0.000438,-0.000042,0.000009,...,0.000604,-0.000049,0.000171,0.000759,-0.000413,-0.000125,-0.000178,0.000102,0.000332,-0.000667
家用电器,0.000161,0.002214,0.001696,-0.002784,-2.624996e-04,0.007796,1.533400e-03,0.003600,0.000047,0.000036,...,-0.000472,-0.000078,0.000306,0.000466,-0.001708,0.000822,-0.001691,-0.000243,-0.000005,0.001689
轻工制造,-0.000126,0.003606,0.001540,-0.003243,6.591435e-07,0.001533,5.712067e-03,0.003398,0.000012,0.000844,...,-0.000198,0.000006,-0.000683,-0.000598,-0.000173,0.001667,-0.000089,0.000201,0.000682,0.002155
休闲服务,-0.000235,0.005025,0.002811,-0.005134,-4.377580e-04,0.003600,3.397543e-03,0.031384,0.000095,0.000896,...,-0.000720,-0.000225,-0.001097,-0.002520,-0.001990,0.003543,-0.001275,-0.000070,0.000739,0.001979
growth,-0.000098,0.000071,0.000018,-0.000010,-4.151527e-05,0.000047,1.208172e-05,0.000095,0.000151,0.000036,...,0.000015,-0.000030,0.000185,0.000559,-0.000201,0.000036,-0.000195,-0.000030,0.000021,0.000059
化工,-0.000042,0.001223,0.001445,-0.002833,9.040031e-06,0.000036,8.440371e-04,0.000896,0.000036,0.003057,...,-0.000009,0.000116,-0.000520,-0.000278,0.002010,0.000164,0.002213,0.000163,0.000296,0.001988


In [38]:
w.T.dot(x).dot(c).dot(x.T).dot(w)

7.26187973550949e-05